# Flow ML Course

## TP 4 : Vertex AI  SDK Python

L'objectif de ce notebook est d'apprendre à utiliser le Model Registry de Vertex AI.

Un Model Registry permeter de créer et de versionner un modèles. Cela permet également de le déployer facilement et de mettre à disposition un endpoint pour requêter le modèle.

En savoir plus [Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction#:~:text=The%20Vertex%20AI%20Model%20Registry%20is%20a%20central,can%20better%20organize%2C%20track%2C%20and%20train%20new%20versions.).


Nous procéderons comme suit :
- Créer et sauver 2 version d'un modèle sur le Model Registry
- Mettre à jour la version du modèle
- Supprimer une version
- Ré-entrainer une autre version du modèle

Nous utiliserons ce type de modèle pour ce TP : [ResNet V2 pretained model](https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5). 

### Model

This tutorial uses a pre-trained image classification model from TensorFlow Hub, which is trained on ImageNet dataset.



In [ ]:
# Install the packages
USER=''
! pip3 install {USER} --upgrade --quiet google-cloud-aiplatform \
                                        tensorflow==2.15.1 \
                                        tensorflow-hub \
                                        tf_keras

In [ ]:
PROJECT_ID = "projet-ia-448520" # "[your-project-id]"  # @param {type:"string"}


# Set the project id
! gcloud config set project {PROJECT_ID}

LOCATION = "us-central1"  # @param {type:"string"}

BUCKET_URI = "gs://cours1bucket"

In [ ]:
import os

import google.cloud.aiplatform as aip
import tensorflow as tf
import tensorflow_hub as hub
import tf_keras

aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

##  Model Registry

Vertex AI Model Registry permet de regrouper plusieurs versions d'un modèle. Cela a pour but de : 

- Suivre les évolutions du model lineage
- Définir la version par défaut mais aussi de déterminer d'autres types de version via les alias (development, staging, production) 
- Déployer un modèle 





## Extraction des modèles

In [ ]:
tfhub_model_v1 = tf_keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model_v1.build([None, 224, 224, 3])

tfhub_model_v1.summary()

In [ ]:
tfhub_model_v2 = tf_keras.Sequential(
    [hub.KerasLayer("https://www.kaggle.com/models/google/resnet-v2/TensorFlow2/50-classification/2")]
)

tfhub_model_v2.build([None, 128, 128, 3])

tfhub_model_v2.summary()

### A vous de jouer
Quelles sont les différences entre les modèles v1 & v2 ?

Votre Réponse : 

### Sauvegarder les modèles artifacts

Nous utiliserons google cloud Storage pour stocker nos modèles.

Utiliser un sous bucket à votre nom pour avoir votre propre répertoire.

In [ ]:
MODEL_V1_DIR = BUCKET_URI + ... + "/model/v1"
tfhub_model_v1.save(MODEL_V1_DIR)

MODEL_V2_DIR = BUCKET_URI + ... + "/model/v2"
tfhub_model_v2.save(MODEL_V2_DIR)

## The Deploy Image

Image de python permettant de faire tourner le modèle.

Ici une image spécifique à tensorflow pour usage CPU

In [ ]:
TF = "2.13".replace(".", "-")

DEPLOY_VERSION = "tf2-cpu.{}".format(TF) # Possibility to use GPU 


DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_VERSION)

### Upload version 1 of the TensorFlow Hub model to a Vertex AI model resource

Next, upload the first version of the model (`MODEL_DIR_V1`) as a model resource in the Vertex AI Model Registry, with the additional following parameters:

- `is_default_version`: Whether this is the default version for the model resource.
- `version_ailiases`: User defined list of alternative alias names for the model version, such as `production`.
- `version_description`: User description of the model version.

When the first model version is created in the Vertex AI Model Registry, the property `version_id` is automatically set to 1.

In [ ]:
your_name = "test"

model_v1 = aip.Model.upload(
    display_name=f"model_course_1_4_{your_name}",
    artifact_uri=MODEL_V1_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    is_default_version=True,
    version_aliases=["v1"],
    version_description=..., # Add a description
)

print(model_v1)

### Upload version 2 of the TensorFlow Hub model to a Vertex AI model resource

Next, upload the second version of the model (`MODEL_DIR_V1`) as a model resource in the Vertex AI Model Registry, with the additional following parameters:

- `parent_model`: The existing model resource for which to add this model as the next model version.
- `is_default_version`: Whether this is the default version for the model resource. In this example, you change from the default from the first version to the second version of the model.
- `version_ailiases`: User defined list of alternative alias names for the model version, such as `production`.
- `version_description`: User description of the model version.

When a subsequent model version is created in the Vertex AI Model Registry, the property `version_id` is automatically incremented. In this example, it's set to 2 (2nd version).

In [ ]:
model_v2 = # Upload the model with parameter : parent_model=model_v1.resource_name

print(model_v2)

### Get all versions of the parent model

Next, list all the versions of a parent model using the `version_registry.list_versions()`.

In [ ]:
versions = model_v1.versioning_registry.list_versions()
for version in versions:
    print(version)

### Get all versions of a non-parent model

You repeat the same, but this time you use a non-parent model. As you can see, you get all the versions regardless if the model you specified is the parent or non-parent model.

In [ ]:
versions = model_v2.versioning_registry.list_versions()
for version in versions:
    print(version)

### Listing a model resource

The remaining Vertex AI SDK methods relating to a model resource automatically use the default version of the model resource.

Next, use the `list()` method with a filter to get the model resources you created above. In this example, there are two versions. Version 2 is set as the default version, so list() only returns the information on version 2 (default version).

In [ ]:
models = aip.Model.list(filter=f"display_name=model_course_1_4_{your_name}")
print("Number of models:", len(models))
print("Version ID:", models[0].version_id)

model = models[0]

### Change the default model version

Next, change which version of the model resource is the default model version using the `versioning_registry.add_version_aliases()` method, with the following parameters:

- `version`: Which version, specified by the version_id, this operation applies to.
- `new_aliases`: The additional aliases to assign to the specified version.

In this example, you refer to the special alias `default` to change this model (version 1) as the default.

Next, use the `list()` method which now returns version 1 instead of version 2 of the model.

In [ ]:
model_v2.versioning_registry.add_version_aliases(new_aliases=["default"], version="1")

models = aip.Model.list(filter=f"display_name=model_course_1_4_{your_name}")
print("Number of models:", len(models))
print("Version ID:", models[0].version_id)

model = models[0]

## Creating an endpoint resource

You create an endpoint resource using the `Endpoint.create()` method. At a minimum, you specify the display name for the endpoint. Optionally, you can specify the project and location (region); otherwise the settings are inherited by the values you set when you initialized the Vertex AI SDK with the `init()` method.

In this example, the following parameters are specified:

- `display_name`: A human readable name for the endpoint resource.
- `project`: Your project ID.
- `location`: Your region.
- `labels`: (optional) User defined metadata for the endpoint in the form of key/value pairs.

This method returns an endpoint object.

Learn more about [Vertex AI endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api).

In [ ]:
endpoint = aip.Endpoint.create(
    display_name=f"endpoint_course_1_4_{your_name}",
    project=PROJECT_ID,
    location=LOCATION,
    labels={"your_key": "your_value"},
)

print(endpoint)

## The Compute

In [ ]:
DEPLOY_COMPUTE = "n1-standard-4"
print("Train machine type", DEPLOY_COMPUTE)

## Deploying model resources to an endpoint resource.

You can deploy one of more Vertex AI model resource instances to the same endpoint. Each Vertex AI model resource that is deployed has its own deployment container for the serving binary. 

*Note:* For this example, you specified the deployment container for the TFHub model in the previous step of uploading the model artifacts to a Vertex AI model resource.

### Deploying a single endpoint resource

In the next example, you deploy the default version of the Vertex AI model resource to a Vertex AI endpoint resource. The Vertex AI model resource already has defined for it the deployment container image. To deploy, you specify the following additional configuration settings:

- The machine type.
- The (if any) type and number of GPUs.
- Static, manual or auto-scaling of VM instances.

In this example, you deploy the model with the minimal amount of specified parameters, as follows:

- `model`: The model resource.
- `deployed_model_displayed_name`: The human readable name for the deployed model instance.
- `machine_type`: The machine type for each VM instance.

Do to the requirements to provision the resource, this may take upto a few minutes.

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name=f"endpoint_course_1_4_{your_name}",
    machine_type=DEPLOY_COMPUTE,
)

print(endpoint)

#### Get information on the deployed model

You can get the deployment settings of the deployed model from the endpoint resource configuration data `gca_resource.deployed_models`. In this example, only one model is deployed -- hence the reference to the subscript `[0]`.

In [ ]:
endpoint.gca_resource.deployed_models[0]

### Undeploy model resource from endpoint resource

When a model resource is deployed to an endpoint resource, the deployed model resource instance is assigned an ID -- commonly referred to as the deployed model ID.

You can undeploy a specific model resource instance with the `undeploy()` method, with the following parameters:

- `deployed_model_id`: The ID assigned to the deployed model.

In [ ]:
deployed_model_id = endpoint.gca_resource.deployed_models[0].id
print(deployed_model_id)

endpoint.undeploy(deployed_model_id)

### Deleting a model version

To delete a version of a model from the Vertex AI Model Registry, you use the `versioning_registry.delete_version()` method, with the following parameters:

- `version`: Can be either the `version_id` or an alias from `version_alias`.

*Note:* You cannot delete the default model version.

In [ ]:
# Deletes a specific model version, as long as it isn't the default version.
model.versioning_registry.delete_version(...) # Choose the version


# Verify that this version is deleted
versions = model.versioning_registry.list_versions()
for version in versions:
    print(version)

### Deleting a model resource

When you delete a model resource, all the associated versions are deleted.

In [ ]:
model.delete()